In [1]:
import torch
import torchvision
import os
import numpy
import collections

In [2]:
class Convolutional(torch.nn.Module):
    def __init__(self, K, O): # K = hidden units / O = outputs
        super(Convolutional, self).__init__()
        # Convolutional Layers
        self.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(5,5), stride=1, padding=2)
        self.conv2 = torch.nn.Conv2d(in_channels=self.conv1.out_channels, out_channels=32, kernel_size=(5,5), stride=1, padding=2)
        self.pool = torch.nn.MaxPool2d(kernel_size(2,2), stride=2)
        # fully connected layers
        self.fc1 = torch.nn.Linear(7*7*32, K, bias=True)
        self.fc2 = torch.nn.Linear(K, O)
        
        self.activation = torch.nn.Sigmoid()
        self.bn = torch.nn.BatchNorm2d(self.conv2.out_channels)
        
    
    def forward(self, x):
        a = self.activation(self.pool(self.conv1(x)))
        # a = self.activation(self.bn(self.pool(self.conv2(a)))) with batch normalization
        a = self.activation(self.pool(self.conv2(a)))
        a = torch.flatten(a, 1)
        return self.fc2(self.activation(self.fc1(a)))

In [3]:
train_adversarial = False
train_with_noise = False

filename = f"Results_{'noise' if train_with_noise else 'adv' if train_adversarial else 'none'}_bn.txt"

torch.manual_seed(42)

transform = torchvision.transforms.ToTensor()
train_set = torchvision.datasets.MNIST(root="data/MNIST",train=True, download=True,transform=transform)
test_set = torchvision.datasets.MNIST(root="data/MNIST",train=False, download=True,transform=transform)

# loaders
train_loader = torch.utils.data.DataLoader(train_set, shuffle=True, batch_size=128)
test_loader = torch.utils.data.DataLoader(test_set, shuffle=False, batch_size=100)

Processing...
Done!
